# Création des resources RDF des mentions de parcelles

In [747]:
import pandas as pd
import numpy as np
import uuid
import re
from rdflib import Graph, Literal, Namespace, RDF, URIRef, BNode
from rdflib.namespace import XSD, DCTERMS, PROV, SKOS, RDFS
from functions import *

## 1. Lecture des données

### Matrices

In [748]:
COMMUNE = 'Gentilly'
matrices_metada = {
    "MAT_1813": {
        "PLAN": "1811",
        "MATRICE_ID": "MAT_B_NB_1813",
        "MATRICE_START": "1813",
        "MATRICE_END": "1835"
    },
    "MAT_1836": {
        "PLAN": "1811",
        "MATRICE_ID": "MAT_NB_1836",
        "MATRICE_START": "1836",
        "MATRICE_END": "1847"
    },
    "MAT_1848": {
        "PLAN": "1845",
        "MATRICE_ID": "MAT_NB_1848",
        "MATRICE_START": "1848",
        "MATRICE_END": "1860"
    }
}


In [749]:
OUTPUT_FOLDER_PATH = "/home/STual/KG-cadastre/data/rdf"

PATH = "/home/STual/KG-cadastre/data/gentilly/MAT_1813.csv"
mat1813 = pd.read_csv(PATH,header=0)
PATH = "/home/STual/KG-cadastre/data/gentilly/MAT_1836.csv"
mat1836 = pd.read_csv(PATH,header=0)
PATH = "/home/STual/KG-cadastre/data/gentilly/MAT_1848.csv"
mat1848 = pd.read_csv(PATH,header=0)

In [750]:
mat1813['registre'] = 'MAT_1813'
mat1836['registre'] = 'MAT_1836'
mat1848['registre'] = 'MAT_1848'

matrices = pd.concat([mat1813, mat1836, mat1848])
matrices = matrices.reset_index(drop=True)
print(matrices.columns)

Index(['ID', 'UUID', 'Type_CF', 'Num_Folio', 'Alt_Num_CF', 'Groupe CF',
       'Ordre de lecture', 'Voie', 'Num_Voie', 'Image', 'Section_clean',
       'Parcelle_clean', 'Lieu-dit_transcript', 'Lieu-dit_clean',
       'Propriétaires_transcript', 'Nature_transcript', 'Nature_clean',
       'Nature_treated', 'Date entrée', 'Date entrée_treated', 'Date sortie',
       'Date sortie_treated', 'Tiré de', 'Tiré de_treated', 'Porté à',
       'Porté à_treated', 'Ligne barrée ?', 'CF rayé ?', 'Spécification',
       'Commentaire', 'Cote liée', 'registre'],
      dtype='object')


### Propriétaires

In [751]:
import json 
import pandas as pd
#open three json files
with open("/home/STual/KG-cadastre/data/gentilly/structured_owners_ok.json") as f:
    data_owner_ok = json.load(f)
with open("/home/STual/KG-cadastre/data/gentilly/structured_owners_nok1.json") as f:
    data_owner_nok1 = json.load(f)
with open("/home/STual/KG-cadastre/data/gentilly/structured_owners_nok2.json") as f:
    data_owner_nok2 = json.load(f)

#concatenate the three json files
data_owners = data_owner_ok + data_owner_nok1 + data_owner_nok2

#read as df
owners_df = pd.DataFrame(data_owners)

In [752]:
oregistre = []
otype_folio = []
ofolio = []
o_groupe_cf = []
o_transcription = []

for row in owners_df.iterrows():
    cell_info = row[1]['cell']
    oregistre.append(cell_info['registre'])
    otype_folio.append(cell_info['type_folio'])
    ofolio.append(cell_info['folio'])
    o_groupe_cf.append(cell_info['groupe_cf'])
    o_transcription.append(cell_info['transcription'])

owners_df['registre'] = oregistre
owners_df['type_folio'] = otype_folio
owners_df['folio'] = ofolio
owners_df['groupe_cf'] = o_groupe_cf
owners_df['transcription'] = o_transcription


In [753]:
owners_df

,cell,owners,changes,registre,type_folio,folio,groupe_cf,transcription
0,"{'registre': 'MAT_1813', 'type_folio': 'Bâti',...","[{'owner-id': 1, 'owner-lastname': 'Louves', '...",[],MAT_1813,Bâti,114,1,Louves
1,"{'registre': 'MAT_1813', 'type_folio': 'Bâti',...","[{'owner-id': 1, 'owner-lastname': 'Hardon', '...",[],MAT_1813,Bâti,82,1,"Hardon, Bourgeois ~~nourisseur~~→à Paris"
2,"{'registre': 'MAT_1813', 'type_folio': 'Bâti',...",NaN,NaN,MAT_1813,Bâti,64,1,Faipot François→m↑d↓ de vin b↑re↓ de fontaineb...
3,"{'registre': 'MAT_1813', 'type_folio': 'Non bâ...","[{'owner-id': 1, 'owner-lastname': 'Besson', '...","[{'change-order': 1, 'owner-before': 1, 'owner...",MAT_1813,Non bâti,23bis,1,~~Besson Jard↑e↓→Fleuriste 1832→Lecoq Jean b↑t...
4,"{'registre': 'MAT_1813', 'type_folio': 'Non bâ...",NaN,NaN,MAT_1813,Non bâti,46bis,1,Bizouard
...,...,...,...,...,...,...,...,...
184,"{'registre': 'MAT_1836', 'type_folio': 'Bâti',...","[{'owner-id': 1, 'owner-lastname': 'Marie Adam...",[],MAT_1836,Bâti,442,6,marie adam→Piétri
185,"{'registre': 'MAT_1836', 'type_folio': 'Bâti',...","[{'owner-id': 1, 'owner-lastname': 'Marie Adam...",[],MAT_1836,Bâti,442,6,marie adam→Piétri
186,"{'registre': 'MAT_1836', 'type_folio': 'Non Bâ...","[{'owner-id': 1, 'owner-lastname': 'Gérard', '...",[],MAT_1836,Non Bâti,708,1,Gérard→route de paris 11
187,"{'registre': 'MAT_1848', 'type_folio': 'Non Bâ...","[{'owner-id': 1, 'owner-lastname': 'Biermann',...",[],MAT_1848,Non Bâti,836,1,"Biermann, Corroyeur à Paris"


### 1.1. Création des pages
- rdf:type rico:Instanciation : instance numérisée d'une page de registre
- rdf:type rico:Record => concept de la page de registre, fait le lien avec le registre (concept, RecordSet)

In [754]:
#select distinct values in th colum Image
images = matrices[['registre','Image']].drop_duplicates()
images

,registre,Image
0,MAT_1813,FRAD094_3P_000255_01_0586
1,MAT_1813,FRAD094_3P_000255_01_0588
2,MAT_1813,FRAD094_3P_000255_01_0579
4,MAT_1813,FRAD094_3P_000255_01_0571
5,MAT_1813,FRAD094_3P_000255_01_0015
...,...,...
415,MAT_1848,FRAD094_3P_000264_01_0216
418,MAT_1848,FRAD094_3P_000264_01_0217
425,MAT_1848,FRAD094_3P_000264_01_0218
427,MAT_1848,FRAD094_3P_000264_01_0219


In [755]:
# Create a new RDF graph
g = Graph()

baseuri = Namespace("http://data.ign.fr/id/source/")
mlclasse = Namespace("http://data.ign.fr/id/codes/cadastre/mlClasse/")
g.bind('source', baseuri)
g.bind('mlclasse', mlclasse)

# Define the namespaces
cad = Namespace("http://data.ign.fr/def/cadastre#")
add = Namespace("http://rdf.geohistoricaldata.org/def/address#")
rico = Namespace("https://www.ica.org/standards/RiC/ontology#")
fpo = Namespace("https://github.com/johnBradley501/FPO/raw/master/fpo.owl#")
time = Namespace("http://www.w3.org/2006/time#")

g.bind('cad', cad)
g.bind('add', add)
g.bind('rico', rico)
g.bind('fpo', fpo)
g.bind('time',time)

for index, row in images.iterrows():
    img = row['Image']
    MATRICE_ID = matrices_metada[row['registre']]["MATRICE_ID"]
    json = parse_record_id(img)
    subject_uri = URIRef(baseuri + f"{img}")
    g.add((subject_uri, RDF.type, rico.Instanciation))
    g.add((subject_uri, rico.identifier, Literal(img)))
    mlClasseNode = BNode()
    g.add((subject_uri, cad.hasClasse, mlClasseNode))
    g.add((mlClasseNode, cad.hasClasseValue, URIRef(mlclasse + f"MATMainTable")))
    g.add((mlClasseNode, PROV.wasGeneratedBy, URIRef(f"http://data.ign.fr/id/codes/cadastre/activity/0001")))
    folder_end = img.rfind('_')
    g.add((subject_uri,rico.isOrWasDigitalInstanciationOf,URIRef(baseuri + f"{json['departement']}_{COMMUNE}_{img}_page")))

    subject_uri_record = URIRef(baseuri + f"{json['departement']}_{COMMUNE}_{row['Image']}_page")
    g.add((subject_uri_record, RDF.type, rico.Record))
    g.add((subject_uri_record, rico.isOrWasIncludedIn, URIRef(baseuri + f"{json['departement']}_{COMMUNE}_{MATRICE_ID}")))

print(g.serialize(format='turtle'))
#write g into a .ttl file
g.serialize(destination=f"{OUTPUT_FOLDER_PATH}/{COMMUNE}_sources_pages.ttl", format='turtle')

@prefix cad: <http://data.ign.fr/def/cadastre#> .
@prefix mlclasse: <http://data.ign.fr/id/codes/cadastre/mlClasse/> .
@prefix prov: <http://www.w3.org/ns/prov#> .
@prefix rico: <https://www.ica.org/standards/RiC/ontology#> .
@prefix source: <http://data.ign.fr/id/source/> .

source:FRAD094_3P_000255_01_0015 a rico:Instanciation ;
    cad:hasClasse [ cad:hasClasseValue mlclasse:MATMainTable ;
            prov:wasGeneratedBy <http://data.ign.fr/id/codes/cadastre/activity/0001> ] ;
    rico:identifier "FRAD094_3P_000255_01_0015" ;
    rico:isOrWasDigitalInstanciationOf source:94_Gentilly_FRAD094_3P_000255_01_0015_page .

source:FRAD094_3P_000255_01_0032 a rico:Instanciation ;
    cad:hasClasse [ cad:hasClasseValue mlclasse:MATMainTable ;
            prov:wasGeneratedBy <http://data.ign.fr/id/codes/cadastre/activity/0001> ] ;
    rico:identifier "FRAD094_3P_000255_01_0032" ;
    rico:isOrWasDigitalInstanciationOf source:94_Gentilly_FRAD094_3P_000255_01_0032_page .

source:FRAD094_3P_00025

<Graph identifier=N43c6b7e1f59c40c196492558bf79eda3 (<class 'rdflib.graph.Graph'>)>

### 1.2 Folios
- Pré-traitement des colonnes *Num_Folio*, *Tiré de* et *Porté à*
- Création des objets "Folios" à partir de la colonne *Num_Folio* et des colonnes *Tiré de* et *Porté à* (manquants)
- Création des objets spéciaux mentionnés dans les colonnes destinées aux folios (reste, construction nouvelle, ruine etc)

#### Pré-traitement

In [756]:
from functions import parse_record_id, cleanNumFolio

#Clean columns Num_Folio, Tire_de, Porte_a
clean_folio, clean_tire_de, clean_porte_a = [], [], []
symbols = [",", "→", "."," ",";","&"]

for index, row in matrices.iterrows():
    clean_folio.append(cleanNumFolio(row["Num_Folio"],symbols))
    clean_tire_de.append(cleanNumFolio(row["Tiré de_treated"],symbols))
    clean_porte_a.append(cleanNumFolio(row["Porté à_treated"],symbols))

# Create new columns containing the cleaned values
matrices['Num_Folio_clean'] = clean_folio
matrices['Tire_de_clean'] = clean_tire_de
matrices['Porte_a_clean'] = clean_porte_a

matrices['Num_Folio_clean'] = matrices['Num_Folio_clean'].astype(str)

['236↑4↓', '361', '258', '166', '235↑2↓', '138', '357', '440']
['248', 'additionconstructionsv']
['249', '249']
['288', '', '433']
['443', '443']
['450', '443', '453', '968']
['449', '968']
['968', '442']
['442', '449', '439', '450', '']
['836', '403']


In [757]:
print(clean_porte_a)

['EMPTY', '82', 'EMPTY', 'EMPTY', 'EMPTY', '156', '156', '192bis', '280bis', 'EMPTY', 'EMPTY', '34ter', 'EMPTY', 'EMPTY', 'EMPTY', 'EMPTY', 'EMPTY', 'EMPTY', 'EMPTY', 'EMPTY', 'EMPTY', 'EMPTY', 'EMPTY', 'EMPTY', 'EMPTY', 'EMPTY', 'EMPTY', 'EMPTY', 'EMPTY', 'EMPTY', 'EMPTY', 'EMPTY', 'EMPTY', 'EMPTY', '124bis', 'EMPTY', 'EMPTY', '192bis', '247ter', 'voiepubliquesv', 'EMPTY', 'EMPTY', '46bis', 'EMPTY', 'EMPTY', 'EMPTY', 'EMPTY', '211bis', '107↑2↓', 'EMPTY', 'doubleemploisv', 'EMPTY', 'EMPTY', '269↑2↓', '138', '236ter', 'EMPTY', 'EMPTY', 'EMPTY', 'EMPTY', 'EMPTY', 'EMPTY', '361', 'EMPTY', 'EMPTY', 'EMPTY', '236↑4↓;361;258;166;235↑2↓;138;357;440', 'EMPTY', 'EMPTY', 'EMPTY', 'EMPTY', '249', 'EMPTY', 'EMPTY', '248', 'EMPTY', '249;249', 'EMPTY', 'EMPTY', '249↑16↓', '249↑16↓', 'EMPTY', 'EMPTY', 'EMPTY', 'EMPTY', 'EMPTY', 'EMPTY', 'EMPTY', 'EMPTY', 'EMPTY', 'EMPTY', 'demolitionsv', 'EMPTY', 'EMPTY', 'demolitionsv', '288;433', 'EMPTY', 'EMPTY', 'EMPTY', 'augmentationsv', 'EMPTY', 'EMPTY', 'EMPTY

In [758]:
#Using matrices, create new df named folios containing all lines of matrices where register = MAT_1836 and MAT_1848. For register=MAT_1813, remove the lines where type_CF = "Bâti"
folios = matrices[(matrices['registre'] == 'MAT_1836') | (matrices['registre'] == 'MAT_1848') | ((matrices['registre'] == 'MAT_1813') & (matrices['Type_CF'] != 'Bâti'))]
folios.reset_index(drop=True)

,ID,UUID,Type_CF,Num_Folio,Alt_Num_CF,Groupe CF,Ordre de lecture,Voie,Num_Voie,Image,...,Porté à_treated,Ligne barrée ?,CF rayé ?,Spécification,Commentaire,Cote liée,registre,Num_Folio_clean,Tire_de_clean,Porte_a_clean
0,6,ea4f017e-09d1-4dc3-95b4-ba4bdfc1d520,Non bâti,11,NaN,1,1,NaN,NaN,FRAD094_3P_000255_01_0015,...,156,Non,Oui,NaN,NaN,NaN,MAT_1813,11,EMPTY,156
1,7,ba879995-f51b-4f44-b9d5-0887d48e8352,Non bâti,11,NaN,1,2,NaN,NaN,FRAD094_3P_000255_01_0015,...,156,Non,Oui,NaN,NaN,NaN,MAT_1813,11,EMPTY,156
2,8,24bb5198-1194-4d5b-b351-5606ad1b5c51,Non bâti,11,NaN,1,3,NaN,NaN,FRAD094_3P_000255_01_0015,...,192bis,Oui,Oui,NaN,NaN,NaN,MAT_1813,11,EMPTY,192bis
3,9,5a84cfac-5ae1-4461-a106-d58e3d37b131,Non bâti,23bis,NaN,1,1,NaN,NaN,FRAD094_3P_000255_01_0032,...,280bis,Oui,Oui,NaN,NaN,NaN,MAT_1813,23bis,constructionnouvellesv,280bis
4,10,936342ed-b994-47a4-909a-0215422aec9a,Non bâti,34ter,NaN,1,1,NaN,NaN,FRAD094_3P_000255_01_0044,...,NaN,Non,Oui,NaN,NaN,NaN,MAT_1813,34ter,46ter,EMPTY
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
422,209,6a3147f9-1fa1-46be-adc9-ba4eb0832b55,Non Bâti,1219,NaN,1,1,NaN,NaN,FRAD094_3P_000264_01_0219,...,NaN,Non,Non,NaN,NaN,NaN,MAT_1848,1219,EMPTY,EMPTY
423,210,8dfa4b95-7ef2-41aa-8d0b-59bc7f00ac78,Non Bâti,1219,NaN,1,2,NaN,NaN,FRAD094_3P_000264_01_0219,...,NaN,Non,Non,NaN,NaN,NaN,MAT_1848,1219,EMPTY,EMPTY
424,211,f6323014-f334-44ae-8135-5af77ffd4296,Non Bâti,1220,NaN,1,1,NaN,NaN,FRAD094_3P_000264_01_0220,...,NaN,Non,Non,NaN,NaN,NaN,MAT_1848,1220,EMPTY,EMPTY
425,212,94ffd0e2-2f07-40f0-9606-c54785cb161d,Non Bâti,1220,NaN,1,2,NaN,NaN,FRAD094_3P_000264_01_0220,...,NaN,Non,Non,NaN,NaN,NaN,MAT_1848,1220,EMPTY,EMPTY


In [759]:
#Create a new dataframe from sources with columns "Num_Folio" and "Image" containing only distinct rows
folios_pages = folios[["Num_Folio_clean","Alt_Num_CF","Image","registre"]].drop_duplicates(subset=["Num_Folio_clean","Alt_Num_CF","Image","registre"]).reset_index(drop=True)
display(folios_pages)

,Num_Folio_clean,Alt_Num_CF,Image,registre
0,11,NaN,FRAD094_3P_000255_01_0015,MAT_1813
1,23bis,NaN,FRAD094_3P_000255_01_0032,MAT_1813
2,34ter,NaN,FRAD094_3P_000255_01_0044,MAT_1813
3,46bis,NaN,FRAD094_3P_000255_01_0057,MAT_1813
4,107bis,NaN,FRAD094_3P_000255_01_0125,MAT_1813
...,...,...,...,...
146,1216,NaN,FRAD094_3P_000264_01_0216,MAT_1848
147,1217,NaN,FRAD094_3P_000264_01_0217,MAT_1848
148,1218,NaN,FRAD094_3P_000264_01_0218,MAT_1848
149,1219,NaN,FRAD094_3P_000264_01_0219,MAT_1848


In [760]:
from rdflib import Graph, Literal, Namespace, RDF, URIRef, BNode
from rdflib.namespace import XSD, DCTERMS
import uuid

# Create a new RDF graph
g = Graph()

baseuri = Namespace("http://data.ign.fr/id/source/")
srctypeuri = Namespace("http://data.ign.fr/id/codes/cadastre/sourceType/")
g.bind('source', baseuri)
g.bind('srctype', srctypeuri)
# Define the namespaces
cad = Namespace("http://data.ign.fr/def/cadastre#")
add = Namespace("http://rdf.geohistoricaldata.org/def/address#")
rico = Namespace("https://www.ica.org/standards/RiC/ontology#")
fpo = Namespace("https://github.com/johnBradley501/FPO/raw/master/fpo.owl#")
time = Namespace("http://www.w3.org/2006/time#")

g.bind('cad', cad)
g.bind('add', add)
g.bind('rico', rico)
g.bind('fpo', fpo)
g.bind('time',time)

# Iterate over each row in the DataFrame
for index, row in folios_pages.iterrows():
    json = parse_record_id(row['Image'])
    MATRICE_ID = matrices_metada[row['registre']]["MATRICE_ID"]

    subject_uri = URIRef(baseuri + f"{json['departement']}_{COMMUNE}_{MATRICE_ID}_{str(row['Num_Folio_clean'])}")
    g.add((subject_uri, RDF.type, rico.RecordPart))
    g.add((subject_uri, cad.isSourceType, URIRef(srctypeuri.Folio)))
    g.add((subject_uri, cad.hasNumFolio, Literal(row["Num_Folio_clean"],datatype=XSD.string)))
    g.add((subject_uri, rico.isOrWasConstituentOf,URIRef(baseuri + f"{json['departement']}_{COMMUNE}_{row['Image']}_page")))

    if not pd.isna(row['Alt_Num_CF']):
        g.add((subject_uri, cad.hasAlternativeNumFolio, Literal(int(row["Alt_Num_CF"]),datatype=XSD.string)))

print(g.serialize(format='turtle'))

@prefix cad: <http://data.ign.fr/def/cadastre#> .
@prefix rico: <https://www.ica.org/standards/RiC/ontology#> .
@prefix source: <http://data.ign.fr/id/source/> .
@prefix srctype: <http://data.ign.fr/id/codes/cadastre/sourceType/> .
@prefix xsd: <http://www.w3.org/2001/XMLSchema#> .

source:94_Gentilly_MAT_B_NB_1813_107bis a rico:RecordPart ;
    cad:hasNumFolio "107bis"^^xsd:string ;
    cad:isSourceType srctype:Folio ;
    rico:isOrWasConstituentOf source:94_Gentilly_FRAD094_3P_000255_01_0125_page .

source:94_Gentilly_MAT_B_NB_1813_11 a rico:RecordPart ;
    cad:hasNumFolio "11"^^xsd:string ;
    cad:isSourceType srctype:Folio ;
    rico:isOrWasConstituentOf source:94_Gentilly_FRAD094_3P_000255_01_0015_page .

source:94_Gentilly_MAT_B_NB_1813_122bis a rico:RecordPart ;
    cad:hasNumFolio "122bis"^^xsd:string ;
    cad:isSourceType srctype:Folio ;
    rico:isOrWasConstituentOf source:94_Gentilly_FRAD094_3P_000255_01_0146_page .

source:94_Gentilly_MAT_B_NB_1813_124bis a rico:RecordPa

#### Création des folios issus de "Tiré de" et "Porté à" qui ne sont pas dans la colonne 'Num_Folios'

In [761]:
# Iterate over each row in the DataFrame
for index, row in folios.iterrows():
    if row['Tire_de_clean'] != 'EMPTY':
        ls = row['Tire_de_clean'].split(";")
        for l in ls:
            if any(num.isdigit() for num in l) and 'omission' not in l:
                MATRICE_ID = matrices_metada[row['registre']]["MATRICE_ID"]

                subject_uri = URIRef(baseuri + f"{json['departement']}_{COMMUNE}_{MATRICE_ID}_{str(l)}")
                g.add((subject_uri, RDF.type, rico.RecordPart))
                g.add((subject_uri, cad.isSourceType, URIRef(srctypeuri.Folio)))
                g.add((subject_uri, cad.hasNumFolio, Literal(l,datatype=XSD.string)))

In [762]:
# Iterate over each row in the DataFrame
for index, row in folios.iterrows():
    if row['Porte_a_clean'] != 'EMPTY':
        ls = row['Porte_a_clean'].split(";")
        for l in ls:
            #test if str has digit
            if any(num.isdigit() for num in l) and 'omission' not in l:
                MATRICE_ID = matrices_metada[row['registre']]["MATRICE_ID"]

                subject_uri = URIRef(baseuri + f"{json['departement']}_{COMMUNE}_{MATRICE_ID}_{str(l)}")
                g.add((subject_uri, RDF.type, rico.RecordPart))
                g.add((subject_uri, cad.isSourceType, URIRef(srctypeuri.Folio)))
                g.add((subject_uri, cad.hasNumFolio, Literal(l,datatype=XSD.string)))

In [763]:
g.serialize(destination=f"{OUTPUT_FOLDER_PATH}/{COMMUNE}_sources_folios.ttl", format='turtle')

<Graph identifier=Nfcaa430a6b104dea9fd82018110e8d58 (<class 'rdflib.graph.Graph'>)>

### 1.3 Dates
Une date dans le cadastre correspond généralement une année.

Une cellule contient une date valide si :
* uniquement des chiffres
* 4 chiffres
* valeur située entre l'initialisation et la clôture de la matrice

### 1.4 Adresses
* Création des objets adresses à partir de la colonne *Lieu-dit* (dédoublonnage simple et création des objets)
* URIS : uuid

In [764]:
# Select the column as a new DataFrame
addresses = matrices[['Lieu-dit_clean']].copy()

# Create pre-treated columns
matrices['Lieu-dit_treated'] = matrices['Lieu-dit_clean']
matrices['Lieu-dit_treated'] = matrices['Lieu-dit_treated'].str.lower()
matrices['Lieu-dit_treated'] = matrices['Lieu-dit_treated'].str.replace(r'[^\w\s~]', '', regex=True)
matrices['Lieu-dit_uuid'] = matrices['Lieu-dit_treated'].map(dict(zip(matrices['Lieu-dit_treated'].unique(), [uuid.uuid4() for _ in range(len(matrices['Lieu-dit_treated'].unique()))])))

In [765]:
addresses = matrices[['registre','Lieu-dit_treated']].drop_duplicates().values.tolist()

In [766]:
# Create a list with all unique values of this new column
unique_addresses = matrices[['registre','Section_clean','Lieu-dit_uuid', 'Lieu-dit_treated']].drop_duplicates().values.tolist()
unique_addresses

[['MAT_1813', 'B', UUID('2b541f04-1541-4770-bef0-da174f89cccd'), nan],
 ['MAT_1813',
  'B',
  UUID('4d8a3363-d657-4309-b29e-f1e583386fe8'),
  'les girantiers'],
 ['MAT_1813', 'B', UUID('87c9fb11-0fc2-4af9-acab-0869e2437253'), 'rue thiers'],
 ['MAT_1813',
  'B',
  UUID('174ca1d1-22d3-4fd0-82d5-bbd422e343ec'),
  'les girantins'],
 ['MAT_1813',
  'B',
  UUID('0536494b-652b-407f-b31a-01b77fce5345'),
  'rue thiers n10'],
 ['MAT_1813',
  'B',
  UUID('d20a59b4-22a9-411d-8cc2-28d216bb4b9a'),
  'route de paris'],
 ['MAT_1813',
  'B',
  UUID('4ade3304-8896-4a39-8a6b-71930528f3d5'),
  'bre fontainebleau'],
 ['MAT_1813', 'B', UUID('daba9ac5-5da7-4261-88d4-8d680163116d'), 'lauteis'],
 ['MAT_1813', 'B', UUID('eb0fc480-c02e-4021-9370-4ea4c8c99ac2'), 'girantins'],
 ['MAT_1836',
  'B',
  UUID('626067b6-d839-4c77-bdb6-d9de44b02dfa'),
  'bd extérieur'],
 ['MAT_1836',
  'B',
  UUID('ad64b3c5-5145-4435-8a15-780bddf2b239'),
  'b ditalie 115'],
 ['MAT_1836',
  'B',
  UUID('209e1924-c9a1-473c-86d7-b9402689d99

A ce stade, il faudrait classer les landmarks en fonction de leur type (lieu-dit, rue, etc.) et les ajouter à la base de données. Pour cela, il faudrait créer une nouvelle colonne dans le dataframe contenant les landmarks, avant de les ajouter au graph.


In [767]:
# Create a new RDF graph
g = Graph()

baseuri = Namespace("http://data.ign.fr/id/landmark/")
srcuri = Namespace("http://data.ign.fr/id/source/")
cad_ltype = Namespace("http://data.ign.fr/def/cadastre/landmarkType/")
g.bind('landmark', baseuri)
g.bind('source', srcuri)
g.bind('cad_ltype', cad_ltype)

# Define the namespaces
cad = Namespace("http://data.ign.fr/def/cadastre#")
add = Namespace("http://rdf.geohistoricaldata.org/def/address#")
atype = Namespace("http://rdf.geohistoricaldata.org/def/address/attributeType/")
ltype = Namespace("http://rdf.geohistoricaldata.org/def/address/landmarkType/")
lrtype = Namespace("http://rdf.geohistoricaldata.org/def/address/landmarkRelationType/")
rico = Namespace("https://www.ica.org/standards/RiC/ontology#")
fpo = Namespace("https://github.com/johnBradley501/FPO/raw/master/fpo.owl#")
time = Namespace("http://www.w3.org/2006/time#")
g.bind('cad', cad)
g.bind('add', add)
g.bind('rico', rico)
g.bind('fpo', fpo)
g.bind('time',time)
g.bind("ltype", ltype)
g.bind("lrtype", lrtype)
g.bind("atype", atype)

for elem in unique_addresses:
    if elem[3] != 'nan':
        subject_uri = URIRef(baseuri + str(elem[2]))
        g.add((subject_uri, RDF.type, add.Landmark))
        g.add((subject_uri, add.isLandmarkType, ltype.Undefined))
        g.add((subject_uri, RDFS.label, Literal(elem[3], datatype=XSD.string)))

        lr = BNode()
        g.add((URIRef(lr.n3()), RDF.type, add.LandmarkRelation))
        g.add((URIRef(lr.n3()), add.isLandmarkRelationType, lrtype.Within))
        g.add((URIRef(lr.n3()), add.locatum, subject_uri))
        if elem[1] == 'MAT_1813' or elem[1] == 'MAT_1836':
            g.add((URIRef(lr.n3()), add.relatum, URIRef('http://data.ign.fr/id/landmark/bea20dea-cce0-4f08-8b5a-517423483e54')))
        else:
            g.add((URIRef(lr.n3()), add.relatum, URIRef('http://data.ign.fr/id/landmark/be7e055d-2b9e-4959-bc9b-283f140f5006')))

#print(g.serialize(format='turtle'))
g.serialize(destination=f"{OUTPUT_FOLDER_PATH}/{COMMUNE}_landmarks_lieu_dit.ttl", format='turtle')

<Graph identifier=N62424f79fdc0494896eca0f64908931c (<class 'rdflib.graph.Graph'>)>

### 1.5 Propriétaires

In [768]:
owners_df.columns
owners_df['groupe_cf'] = owners_df['groupe_cf'].astype(str)
matrices['Groupe CF'] = matrices['Groupe CF'].astype(str)

In [769]:
matrices = matrices.merge(owners_df, how='left', right_on=["registre","type_folio","folio",'groupe_cf'], left_on=["registre","Type_CF","Num_Folio","Groupe CF"])
matrices = matrices.drop(["type_folio","folio",'groupe_cf',"transcription","cell"], axis=1).reset_index(drop=True)
matrices['CF_uuid'] = [uuid.uuid4() for _ in range(len(matrices))]
matrices['CF_uuid'] = matrices.groupby(['registre', 'Num_Folio_clean','Type_CF','Groupe CF'])['CF_uuid'].transform(lambda x: uuid.uuid4())
matrices

,ID,UUID,Type_CF,Num_Folio,Alt_Num_CF,Groupe CF,Ordre de lecture,Voie,Num_Voie,Image,...,Cote liée,registre,Num_Folio_clean,Tire_de_clean,Porte_a_clean,Lieu-dit_treated,Lieu-dit_uuid,owners,changes,CF_uuid
0,1,b14e16a7-54a9-472b-9e15-d71d1cde7b39,Bâti,108,NaN,1,1,NaN,NaN,FRAD094_3P_000255_01_0586,...,NaN,MAT_1813,108,EMPTY,EMPTY,NaN,2b541f04-1541-4770-bef0-da174f89cccd,"[{'owner-id': 1, 'owner-lastname': 'Legendre',...",[],92c2883e-f5a6-44c5-baf8-2213360fc76e
1,2,1d704267-987d-4a0a-89bd-a96620da39e5,Bâti,114,NaN,1,1,NaN,NaN,FRAD094_3P_000255_01_0588,...,NaN,MAT_1813,114,EMPTY,82,NaN,2b541f04-1541-4770-bef0-da174f89cccd,"[{'owner-id': 1, 'owner-lastname': 'Louves', '...",[],247c086f-4780-4cc9-b1d5-9a043dd095f4
2,3,622eb82c-1332-4be8-a0fc-805ff7eb3421,Bâti,82,NaN,1,1,NaN,NaN,FRAD094_3P_000255_01_0579,...,NaN,MAT_1813,82,114,EMPTY,NaN,2b541f04-1541-4770-bef0-da174f89cccd,"[{'owner-id': 1, 'owner-lastname': 'Hardon', '...",[],fbb82503-ba83-49eb-8c1e-0b4649042cf0
3,4,03456c67-aff8-49ed-9291-d4c27aa28373,Bâti,82,NaN,1,2,NaN,NaN,FRAD094_3P_000255_01_0579,...,NaN,MAT_1813,82,EMPTY,EMPTY,NaN,2b541f04-1541-4770-bef0-da174f89cccd,"[{'owner-id': 1, 'owner-lastname': 'Hardon', '...",[],fbb82503-ba83-49eb-8c1e-0b4649042cf0
4,5,2ecd32f7-5c98-4a74-b94e-ccd28464bfe0,Bâti,64,NaN,1,1,NaN,NaN,FRAD094_3P_000255_01_0571,...,NaN,MAT_1813,64,EMPTY,EMPTY,NaN,2b541f04-1541-4770-bef0-da174f89cccd,NaN,NaN,1444f2d6-a309-418d-bddc-2cecc18b0599
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
430,209,6a3147f9-1fa1-46be-adc9-ba4eb0832b55,Non Bâti,1219,NaN,1,1,NaN,NaN,FRAD094_3P_000264_01_0219,...,NaN,MAT_1848,1219,EMPTY,EMPTY,bre ditie mon blanche 8a,c52ad1a4-8b22-450c-919e-3c140f78288c,"[{'owner-id': 1, 'owner-lastname': 'Delon', 'i...",[],f0c06272-77de-42f4-bfda-52799de54b67
431,210,8dfa4b95-7ef2-41aa-8d0b-59bc7f00ac78,Non Bâti,1219,NaN,1,2,NaN,NaN,FRAD094_3P_000264_01_0219,...,NaN,MAT_1848,1219,EMPTY,EMPTY,bre ditie mon blanche 8a,c52ad1a4-8b22-450c-919e-3c140f78288c,"[{'owner-id': 1, 'owner-lastname': 'Delon', 'i...",[],f0c06272-77de-42f4-bfda-52799de54b67
432,211,f6323014-f334-44ae-8135-5af77ffd4296,Non Bâti,1220,NaN,1,1,NaN,NaN,FRAD094_3P_000264_01_0220,...,NaN,MAT_1848,1220,EMPTY,EMPTY,bre ditie mon blanche 6,9e918682-d76e-46d4-8917-3d9f4cc82b6d,"[{'owner-id': 1, 'owner-lastname': 'Dufresne',...",[],24527afd-27b9-4ca6-bf65-6b3d63ce183a
433,212,94ffd0e2-2f07-40f0-9606-c54785cb161d,Non Bâti,1220,NaN,1,2,NaN,NaN,FRAD094_3P_000264_01_0220,...,NaN,MAT_1848,1220,EMPTY,EMPTY,bre ditie mon blanche 6,9e918682-d76e-46d4-8917-3d9f4cc82b6d,"[{'owner-id': 1, 'owner-lastname': 'Dufresne',...",[],24527afd-27b9-4ca6-bf65-6b3d63ce183a


* Créer les comptes fonciers
* Associer à chaque compte foncier ses propriétaires (ordonnés dans le temps)
* Associer à chaque compte foncier le landmark (état) qu'il mentionne

In [772]:
# Create a new RDF graph
g = Graph()

baseuri = Namespace("http://data.ign.fr/id/landmark/")
srcuri = Namespace("http://data.ign.fr/id/source/")
cad_ltype = Namespace("http://data.ign.fr/def/cadastre/landmarkType/")
cad_atype = Namespace("http://data.ign.fr/def/cadastre/attributeType/")
srctype = Namespace("http://data.ign.fr/id/codes/cadastre/sourceType/")
mlclasse= Namespace("http://data.ign.fr/id/codes/cadastre/mlClasse/")

g.bind('landmark', baseuri)
g.bind('source', srcuri)
g.bind('cad_ltype', cad_ltype)
g.bind('srctype', srctype)
g.bind('mlclasse', mlclasse)

for index, row in matrices.iterrows():
    json = parse_record_id(row["Image"])
    lineuuid_ = MATRICE_ID + '_' + str(row['ID']) #str(uuid.uuid4())

    subject_uri = URIRef(srcuri + f"{row['CF_uuid']}")
    g.add((subject_uri, RDF.type, rico.RecordPart))
    g.add((subject_uri, cad.isSourceType, URIRef(srctype.CompteFoncier)))
    g.add((subject_uri, rico.hasOrHadConstituent, Literal(srcuri + f"{row['UUID']}")))
    g.add((subject_uri, rico.hasOrWasConstituentOf, URIRef(baseuri + f"{json['departement']}_{COMMUNE}_{MATRICE_ID}_{str(row['Num_Folio_clean'])}")))
    g.add((subject_uri, rico.hasOrHadConstituent, Literal(srcuri + f"{row['CF_uuid']}_mutations")))

    articlemutationuri = URIRef(srcuri + f"{row['CF_uuid']}_mutations")
    g.add((articlemutationuri, RDF.type, rico.RecordPart))
    g.add((articlemutationuri, cad.isSourceType, URIRef(srctype.ArticleDeMutation)))

    ownersattribute = BNode()
    g.add((ownersattribute, RDF.type, add.Attribute))
    g.add((subject_uri, cad.hasCadastreAttribute, ownersattribute))
    g.add((ownersattribute, add.hasAttributeType, URIRef(cad_atype.PlotTaxpayer)))
    print(row['owners'])


    articleclassementuri = URIRef(srcuri + f"{row['UUID']}")
    g.add((articleclassementuri, RDF.type, rico.RecordPart))
    g.add((articleclassementuri, cad.isSourceType, URIRef(srctype.ArticleDeClassement)))

[{'owner-id': 1, 'owner-lastname': 'Legendre', 'owner-address': 'H↑re↓ de →Fontainebleau', 'is-stripped': 'no'}]
[{'owner-id': 1, 'owner-lastname': 'Louves', 'is-stripped': 'no'}]
[{'owner-id': 1, 'owner-lastname': 'Hardon', 'owner-job': 'Bourgeois', 'owner-activity': 'nourisseur', 'owner-address': 'à Paris', 'is-stripped': 'no'}]
[{'owner-id': 1, 'owner-lastname': 'Hardon', 'owner-job': 'Bourgeois', 'owner-activity': 'nourisseur', 'owner-address': 'à Paris', 'is-stripped': 'no'}]
nan
[{'owner-id': 1, 'owner-lastname': 'Bacot', 'owner-firstname': 'David', 'owner-activity': 'couverturier', 'owner-address': 'à Paris', 'is-stripped': 'no'}]
[{'owner-id': 1, 'owner-lastname': 'Bacot', 'owner-firstname': 'David', 'owner-activity': 'couverturier', 'owner-address': 'à Paris', 'is-stripped': 'no'}]
[{'owner-id': 1, 'owner-lastname': 'Bacot', 'owner-firstname': 'David', 'owner-activity': 'couverturier', 'owner-address': 'à Paris', 'is-stripped': 'no'}]
[{'owner-id': 1, 'owner-lastname': 'Besson

### 1.6 Nature
* Associer chaque transcription avec un ou plusieurs types de skos = fait à la main pour l'instant

### 1.X Création des états de parcelles

In [771]:
# Create a new RDF graph
g = Graph()

baseuri = Namespace("http://data.ign.fr/id/landmark/")
srcuri = Namespace("http://data.ign.fr/id/source/")
owneruri = Namespace("http://data.ign.fr/id/owner/")
foliouri = Namespace("http://data.ign.fr/id/folio/")
cad_ltype = Namespace("http://data.ign.fr/def/cadastre/landmarkType/")
cad_atype = Namespace("http://data.ign.fr/def/cadastre/attributeType/")
g.bind('landmark', baseuri)
g.bind('owner', owneruri)
g.bind('source', srcuri)
g.bind('cad_ltype', cad_ltype)
g.bind('cad_atype', cad_atype)

# Define the namespaces
cad = Namespace("http://data.ign.fr/def/cadastre#")
add = Namespace("http://rdf.geohistoricaldata.org/def/address#")
rico = Namespace("https://www.ica.org/standards/RiC/ontology#")
fpo = Namespace("https://github.com/johnBradley501/FPO/raw/master/fpo.owl#")
time = Namespace("http://www.w3.org/2006/time#")

g.bind('cad', cad)
g.bind('add', add)
g.bind('rico', rico)
g.bind('fpo', fpo)
g.bind('time',time)

# Iterate over each row in the DataFrame
for index, row in matrices.iterrows():

    json = parse_record_id(row["Image"])
    lineuuid_ = MATRICE_ID + '_' + str(row['ID']) #str(uuid.uuid4())

    subject_uri = URIRef(baseuri + f"{json['departement']}_{COMMUNE}_{PLAN}_{row['Section_clean']}_{row['Parcelle_clean']}_{lineuuid_}")
    g.add((subject_uri, RDF.type, add.Landmark))
    g.add((subject_uri, add.isLandmarkType, cad_ltype.Plot))
    g.add((subject_uri, DCTERMS.identifier, Literal(row['Section_clean'] + '-' + row['Parcelle_clean'], datatype=XSD.string)))
    #Folios
    g.add((subject_uri, cad.hasNumFolio, URIRef(foliouri + f"{json['departement']}_{COMMUNE}_{MATRICE_ID}_{str(row['Num_Folio_clean'])}")))
    tire_de = row['Tire_de_clean'] = row['Tire_de_clean'].split(';')
    for f in tire_de:
        if any(char.isdigit() for char in str(f)):
            g.add((subject_uri, cad.takenFrom, URIRef(foliouri + f"{json['departement']}_{COMMUNE}_{MATRICE_ID}_{str(f)}")))
        elif f != 'nan':
            g.add((subject_uri, cad.takenFrom, Literal(str(f))))
    porte_a = row['Porte_a_clean'] = row['Porte_a_clean'].split(';')
    for f in porte_a:
        if any(char.isdigit() for char in str(f)):
            g.add((subject_uri, cad.passedTo, URIRef(foliouri + f"{json['departement']}_{COMMUNE}_{MATRICE_ID}_{str(f)}")))
        elif f != 'nan':
            g.add((subject_uri, cad.passedTo, Literal(str(f))))
    #Source
    rowSource = BNode()
    g.add((subject_uri, fpo.sourcedFrom, rowSource))
    g.add((rowSource, RDF.type, fpo.SourceCitation))
    g.add((rowSource, fpo.fromSource, URIRef(srcuri + f"{json['departement']}_{COMMUNE}_{MATRICE_ID}")))
    g.add((rowSource, rico.isComponentOfTransitive, URIRef(srcuri + f"{json['departement']}_{COMMUNE}_{row['Image']}")))
    g.add((rowSource, cad.hasExtractionID, Literal(lineuuid_)))
    g.add((rowSource, PROV.wasGeneratedBy, URIRef(f"http://data.ign.fr/id/codes/cadastre/activity/0002")))
    g.add((URIRef(f"http://data.ign.fr/id/codes/cadastre/activity/0002"), PROV.used, URIRef(srcuri + f"{json['departement']}_{COMMUNE}_{row['Image']}")))
    g.add((rowSource, rico.isOrWasDigitalInstanciation,URIRef(srcuri + f"{json['departement']}_{COMMUNE}_{row['Image']}_{lineuuid_}_area")))
    g.add((rowSource, cad.lineOrderInArea, Literal(row['Ordre de lecture'], datatype=XSD.integer)))

    #Create recordpart
    recordparturi = URIRef(srcuri + f"{json['departement']}_{COMMUNE}_{row['Image']}_{lineuuid_}_area")
    g.add((recordparturi, RDF.type, rico.RecordPart))
    g.add((recordparturi, rico.isOrWasIncludedIn, URIRef(srcuri + f"{json['departement']}_{COMMUNE}_{row['Image']}_page")))
    
    #Address
    if row['Lieu-dit_treated'] != 'nan':
        plotaddress = BNode()
        g.add((subject_uri, add.hasAttribute, plotaddress))
        g.add((plotaddress, add.isAttributeType, cad_atype.PlotAddress))
        plotaddressversion = BNode()
        g.add((plotaddress, add.hasAttributeVersion, plotaddressversion))
        g.add((plotaddressversion, add.isLandmarkRelationType, lrtype.Undefined))
        g.add((plotaddressversion, add.locatum, subject_uri))
        g.add((plotaddressversion, add.relatum, URIRef(baseuri + str(row['Lieu-dit_uuid']))))

    #Owner

    #Nature
    if not pd.isnull(row['Nature']):
        nature = BNode()
        g.add((subject_uri, add.hasNature, nature))
        g.add((nature, RDF.type, add.Nature))
        g.add((nature, RDFS.label, Literal(row['Nature'], datatype=XSD.string)))
    
    #Time
    if not pd.isnull(row['Date entrée']) or not pd.isnull(['Date sortie']):
        hastime = BNode()
        g.add((subject_uri, add.hasTime, hastime))
        g.add((hastime, RDF.type, add.TimeInterval))
        

        if not pd.isna(row['Date entrée']) and not pd.isnull(row['Date entrée']):
            hasbeginning = BNode()
            g.add((hastime, add.hasBeginning, hasbeginning))
            g.add((hasbeginning,RDF.type, add.TimeInstant))
            g.add((hasbeginning, add.timeCalendar, time.Gregorian))
            g.add((hasbeginning, add.timePrecision, time.Year))
            g.add((hasbeginning, add.timeStamp, Literal(row['Date entrée'], datatype=XSD.date)))
        #else:
            #g.add((hastime, add.hasBeginning, Literal(MATRICE_START, datatype=XSD.date)))#date d'ouverture de la matrice
        if not pd.isnull(row['Date sortie']) and row['Date sortie'] != 'nan':
            hasend = BNode()
            g.add((hastime, add.hasBeginning, hasend))
            g.add((hasend,RDF.type, add.TimeInstant))
            g.add((hasend, add.timeCalendar, time.Gregorian))
            g.add((hasend, add.timePrecision, time.Year))
            g.add((hasend, add.timeStamp, Literal(row['Date entrée'], datatype=XSD.date)))
print(g.serialize(format='turtle'))

NameError: name 'PLAN' is not defined